바로 아래 코드는 마지막 최종 코드입니다.

맷플롭립으로 그래프를 그려본 결과 alcohol 특성 quality와 비례하는 것을 알게되었습니다. alcohol의 제곱을 갖는
sqalco 라는 특성을 추가해주었습니다.
주어진 csv파일을 5등분해서 3개분을 training 으로 1개분을 validate로 1개분을 test로 활용할것입니다.
5등분중 어떤 부분이 전체 데이터를 잘 대표하는지 알기 위해서 순열을 사용해서 총 20번 동안 반복문을 돌아
가장 퍼포먼스가 좋은 경우를 골랐습니다.

In [ ]:
import pandas as pd
from autograd import numpy as np
from autograd import grad
from sklearn.preprocessing import StandardScaler
def logistic(x):
    out = 1. / (1. + np.exp(-x))
    return out
def logistic_model(x,params):
    out = logistic(np.dot(x,params[0])+params[1])
    return out
def model_loss(x,true_labels,params,_lambda=1.0):
    pred = logistic_model(x,params)

    loss =  - (np.dot(true_labels, np.log(pred+1e-15)) + np.dot(1.-true_labels, np.log(1.-pred+1e-15))) \
        + _lambda * np.sum(params[0]**2)
    return loss
def classify(x,params):
    probabilities = logistic_model(x,params)
    labels = (probabilities >= 0.5).astype(float)
    return labels
def performance(predictions, answers, beta = 1.0):
    true_idx = (answers == 1)
    false_idx = (answers == 0)

    n_tp = np.count_nonzero(predictions[true_idx] == 1)

    n_fp = np.count_nonzero(predictions[false_idx] == 1)

    n_tn = np.count_nonzero(predictions[false_idx] == 0)

    n_fn = np.count_nonzero(predictions[true_idx] == 0)

    precision = n_tp / (n_tp + n_fp)
    recall = n_tp / (n_tp + n_fn)

    score = ((1.0 + beta**2) * precision * recall / (beta ** 2 * precision + recall))

    accuracy = (n_tp + n_tn) / (n_tp + n_fp + n_tn + n_fn)

    return precision,recall,score,accuracy

In [ ]:
from itertools import permutations as pm

pm_helper = [0,1,2,3,4]

pm_list = pm(pm_helper,2)

sqalco_datas = pd.read_csv("winequality-red.csv")
sqalco_datas=sqalco_datas.append(sqalco_datas.loc[0],ignore_index = True)

#print(sqalco_datas.shape[0])
#sqalco_datas = sqalco_datas.sample(frac=1).reset_index(drop=True)



sqalco_datas["sqalco"] = [0.] * sqalco_datas.shape[0]

for i in range(sqalco_datas.shape[0]):
    sqalco_datas["sqalco"][i] = sqalco_datas["alcohol"][i] ** 2

y_col = "quality"
x_cols = sqalco_datas.columns.drop(["quality"])

for i in range(len(sqalco_datas[y_col])):
    if sqalco_datas[y_col][i] < 6:
        sqalco_datas[y_col][i] = 0.0
    else:
        sqalco_datas[y_col][i] = 1.0

sqalco_datas[x_cols] = StandardScaler().fit_transform(sqalco_datas[x_cols])




num_of_datas = sqalco_datas.shape[0]
num_of_features = len(x_cols)
#print(num_of_datas,num_of_features)

best_scores = 0.0
best_w = None
best_b = None
best_fsc = 0.0
best_acc = 0.0
iters0 = None
iters1 = None

datas_division = np.split(sqalco_datas[x_cols],5,0)
labels_division = np.split(sqalco_datas[y_col],5,0)

assert datas_division[0].shape[0] == labels_division[0].shape[0]
assert datas_division[1].shape[0] == labels_division[1].shape[0]
assert datas_division[2].shape[0] == labels_division[2].shape[0]
assert datas_division[3].shape[0] == labels_division[3].shape[0]
assert datas_division[4].shape[0] == labels_division[4].shape[0]
wtt = 0
for iters in pm_list:
    wtt += 1
    pm_helper = [0,1,2,3,4]

    datas_val=datas_division[iters[0]].values
    datas_test = datas_division[iters[1]].values

    labels_val = labels_division[iters[0]].values
    labels_test = labels_division[iters[1]].values

    pm_helper.remove(iters[0])
    pm_helper.remove(iters[1])
    
    datas_train=np.vstack((datas_division[pm_helper[0]].values,datas_division[pm_helper[1]].values,datas_division[pm_helper[2]].values))


    labels_train=np.hstack((labels_division[pm_helper[0]].values,labels_division[pm_helper[1]].values,labels_division[pm_helper[2]].values))

    

    
    #print(labels_train.shape)
    #print(datas_train.shape)
    assert labels_train.shape[0] == datas_train.shape[0]
    assert labels_val.shape[0] == datas_val.shape[0]
    assert labels_test.shape[0] == datas_test.shape[0]

    gradients = grad(model_loss,argnum = 2)

    w = np.zeros(datas_train.shape[1],dtype=float)

    b=0.

    lr = 1e-5

    change = np.Inf

    i = 0

    old_val_loss = 1e-15

    while change >= 1e-15 and i<10000:#
        grads = gradients(datas_train, labels_train,(w,b))
        w-=(grads[0] * lr)
        b-=(grads[1] * lr)

        val_loss = model_loss(datas_val,labels_val,(w,b))

        pred_labels_val = classify(datas_val, (w,b))
        score = performance(pred_labels_val, labels_val)

        change = np.abs((val_loss - old_val_loss)/old_val_loss)

        i += 1
        old_val_loss = val_loss

        if i% 5000 == 0:
            print("{}...".format(i),end="")
        if wtt%10 and i%500 == 0:
            print("와우!!!")
    if ( score[3] + score[2] ) > best_scores:
        best_b = b
        best_w = w
        best_fsc = score[2]
        best_acc = score[3]
        best_scores = score[3] + score[2]
        iters0 = iters[0]
        iters1 = iters[1]
#print("")
#print("")
#print("Upon optimization stopped:")
#print("    Iterations:", i)
#print("    Validation loss:", val_loss)
#print("    Validation precision:", score[0])
#print("    Validation recall:", score[1])
print("    Validation F-score:", score[2])
print("    Validation Accuracy:", score[3])
print("    Change in validation loss:", change)

In [ ]:
print(best_fsc,best_acc,best_b,best_w)

In [ ]:
tdatas = pd.read_csv("winequality-red_test.csv")
tdatas["sqalco"] = [0.] * tdatas.shape[0]

for i in range(tdatas.shape[0]):
    tdatas["sqalco"][i] = tdatas["alcohol"][i] ** 2

tdatas = StandardScaler().fit_transform(tdatas)
dec = classify(tdatas,(best_w,best_b))

outdatas=pd.read_csv("sample_sub.csv")
for i in range(len(outdatas["quality"])):
    outdatas["quality"][i] = int(dec[i])
outdatas.to_csv("./result2.csv",index=False)

이 아래는 모두 시행착오들 입니다.

In [351]:
import pandas as pd
from autograd import numpy as np
from autograd import grad
from sklearn.preprocessing import StandardScaler


In [352]:
datas = pd.read_csv("winequality-red.csv")
print(f"datas head : {datas.head()}")

datas head :    fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      

In [353]:
print(f"datas information : {datas.info()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1439 entries, 0 to 1438
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1439 non-null   float64
 1   volatile acidity      1439 non-null   float64
 2   citric acid           1439 non-null   float64
 3   residual sugar        1439 non-null   float64
 4   chlorides             1439 non-null   float64
 5   free sulfur dioxide   1439 non-null   float64
 6   total sulfur dioxide  1439 non-null   float64
 7   density               1439 non-null   float64
 8   pH                    1439 non-null   float64
 9   sulphates             1439 non-null   float64
 10  alcohol               1439 non-null   float64
 11  quality               1439 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 135.0 KB
datas information : None


multiple logistic regression will be good!!

In [354]:
y_col = "quality"
x_cols = datas.columns.drop(["quality"])
print(x_cols)


Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')


In [355]:
print(datas[y_col].describe())

count    1439.000000
mean        5.642113
std         0.804494
min         3.000000
25%         5.000000
50%         6.000000
75%         6.000000
max         8.000000
Name: quality, dtype: float64


0-5 quality means 0
6-10 quality means 1

In [356]:
for i in range(len(datas[y_col])):
    if datas[y_col][i] < 6:
        datas[y_col][i] = 0.0
    else:
        datas[y_col][i] = 1.0

C:\Users\USER\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\USER\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


make functions

In [357]:
def logistic(x):
    out = 1. / (1. + np.exp(-x))
    return out

In [358]:
def logistic_model(x,params):
    out = logistic(np.dot(x,params[0])+params[1])
    return out

In [359]:
def model_loss(x,true_labels,params,_lambda=1.0):
    pred = logistic_model(x,params)

    loss =  - (np.dot(true_labels, np.log(pred+1e-15)) + np.dot(1.-true_labels, np.log(1.-pred+1e-15))) \
        + _lambda * np.sum(params[0]**2)
    return loss

normalize x datas by z-score using sklearn module

In [360]:
print(datas[x_cols].describe().loc[["min","max"]])
print(datas[x_cols].head())

     fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
min            4.6              0.12          0.0             0.9      0.012   
max           15.9              1.58          1.0            15.5      0.611   

     free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
min                  1.0                   6.0  0.99007  2.74       0.33   
max                 72.0                 289.0  1.00369  4.01       2.00   

     alcohol  
min      8.4  
max     14.9  
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur d

In [361]:
datas[x_cols] = StandardScaler().fit_transform(datas[x_cols])

In [362]:
print(datas[x_cols].describe().loc[["min","max"]])
print(datas[x_cols].head())

     fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
min      -2.220257         -2.255136    -1.429020       -1.208230  -1.583605   
max       4.263980          5.888884     3.700339        9.535715  10.755652   

     free sulfur dioxide  total sulfur dioxide   density        pH  sulphates  \
min            -1.398770             -1.232906 -3.592635 -3.632006   -1.87844   
max             5.424708              7.284082  3.617157  4.568737    7.65551   

      alcohol  
min -1.856650  
max  4.165187  
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0      -0.613544          0.980159    -1.429020       -0.472343  -0.265220   
1      -0.384013          1.984217    -1.429020        0.042778   0.187975   
2      -0.384013          1.314845    -1.223846       -0.177988   0.064376   
3       1.566996         -1.362641     1.443421       -0.472343  -0.285820   
4      -0.613544          0.980159    -1.429020       -0.472343  -0.265220   

In [363]:
num_of_datas = datas.shape[0]
num_of_features = len(x_cols)
print(num_of_datas,num_of_features)

1439 11


In [364]:
num_of_val = int(num_of_datas * 0.2)

num_of_test = int(num_of_datas * 0.2)

num_of_train = num_of_datas - num_of_val - num_of_test
print(f"number of validation : {num_of_val}")
print(f"number of testing : {num_of_test}")
print(f"number of trainning : {num_of_train}")
print(f"sum : {num_of_train + num_of_val + num_of_test}")

number of validation : 287
number of testing : 287
number of trainning : 865
sum : 1439


In [365]:
datas_division = np.split(datas[x_cols],[num_of_val, num_of_val + num_of_test],0)
datas_val = datas_division[0].values
datas_test = datas_division[1].values
datas_train = datas_division[2].values

labels_division = np.split(datas[y_col],[num_of_val, num_of_val + num_of_test],0)
labels_val = labels_division[0].values
labels_test = labels_division[1].values
labels_train = labels_division[2].values

In [366]:
assert labels_train.shape[0] == datas_train.shape[0]
assert labels_val.shape[0] == datas_val.shape[0]
assert labels_test.shape[0] == datas_test.shape[0]

In [367]:
def classify(x,params):
    probabilities = logistic_model(x,params)
    labels = (probabilities >= 0.5).astype(float)
    return labels

In [368]:
def performance(predictions, answers, beta = 1.0):
    true_idx = (answers == 1)
    false_idx = (answers == 0)

    n_tp = np.count_nonzero(predictions[true_idx] == 1)

    n_fp = np.count_nonzero(predictions[false_idx] == 1)

    n_tn = np.count_nonzero(predictions[false_idx] == 0)

    n_fn = np.count_nonzero(predictions[true_idx] == 0)

    precision = n_tp / (n_tp + n_fp)
    recall = n_tp / (n_tp + n_fn)

    score = ((1.0 + beta**2) * precision * recall / (beta ** 2 * precision + recall))

    accuracy = (n_tp + n_tn) / (n_tp + n_fp + n_tn + n_fn)

    return precision,recall,score,accuracy

In [369]:
gradients = grad(model_loss,argnum = 2)

w = np.zeros(datas_train.shape[1],dtype=float)

b=0.

In [370]:
pred_labels_test = classify(datas_test,(w,b))
perf = performance(pred_labels_test,labels_test)

print("Initial precision: {:.1f}%".format(perf[0]*100))
print("Initial recall: {:.1f}%".format(perf[1]*100))
print("Initial F-score: {:.1f}%".format(perf[2]*100))
print("Initial Accuracy: {:.1f}%".format(perf[3]*100))

Initial precision: 64.1%
Initial recall: 100.0%
Initial F-score: 78.1%
Initial Accuracy: 64.1%


In [371]:
lr = 1e-5

change = np.Inf

i = 0

old_val_loss = 1e-15

while change >= 1e-15 and i<10000:
    grads = gradients(datas_train, labels_train,(w,b))
    w-=(grads[0] * lr)
    b-=(grads[1] * lr)

    val_loss = model_loss(datas_val,labels_val,(w,b))

    pred_labels_val = classify(datas_val, (w,b))
    score = performance(pred_labels_val, labels_val)

    change = np.abs((val_loss - old_val_loss)/old_val_loss)

    i += 1
    old_val_loss = val_loss

    if i% 50 == 0:
        print("{}...".format(i),end="")
print("")
print("")
print("Upon optimization stopped:")
print("    Iterations:", i)
print("    Validation loss:", val_loss)
print("    Validation precision:", score[0])
print("    Validation recall:", score[1])
print("    Validation F-score:", score[2])
print("    Validation Accuracy:", score[3])
print("    Change in validation loss:", change)

50...100...150...200...250...300...350...400...450...500...550...600...650...700...750...800...850...900...950...1000...1050...1100...1150...1200...1250...1300...1350...1400...1450...1500...1550...1600...1650...1700...1750...1800...1850...1900...1950...2000...2050...2100...2150...2200...2250...2300...2350...2400...2450...2500...2550...2600...2650...2700...2750...2800...2850...2900...2950...3000...3050...3100...3150...3200...3250...3300...3350...3400...3450...3500...3550...3600...3650...3700...3750...3800...3850...3900...3950...4000...4050...4100...4150...4200...4250...4300...4350...4400...4450...4500...4550...4600...4650...4700...4750...4800...4850...4900...4950...5000...5050...5100...5150...5200...5250...5300...5350...5400...5450...5500...5550...5600...5650...5700...5750...5800...5850...5900...5950...6000...6050...6100...6150...6200...6250...6300...6350...6400...6450...6500...6550...6600...6650...6700...6750...6800...6850...6900...6950...7000...7050...7100...7150...7200...7250...7300.

do it! test


아래 부터는 람다나 러닝레이트같은 파라미터를 조절하거나 pca 모듈로 dimension을 줄이기, k-fold cross validation,모델 재정의
를 하고 있습니다. 마지막 코드가 최종 코드 입니다!

In [378]:
tdatas = pd.read_csv("winequality-red_test.csv")

tdatas = StandardScaler().fit_transform(tdatas)
dec = classify(tdatas,(w,b))

In [381]:
outdatas=pd.read_csv("sample_sub.csv")
for i in range(len(outdatas["quality"])):
    outdatas["quality"][i] = int(dec[i])
outdatas.to_csv("./result1.csv",index=False)

C:\Users\USER\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


I will define other model again because the result is not good.

lambda ,leaning rate, data distribution ...

In [386]:
pltdatas = pd.read_csv("winequality-red.csv")

In [ ]:
import matplotlib.pyplot as plt
import time
#plt.figure(figsize=(5,5))

for i in range(11):
    plt.plot(pltdatas[x_cols[i]],pltdatas[y_col],"bo")
    plt.title(x_cols[i])
    plt.show()



1. I think the alcohol feature is proportional to the quality.
2. And the rest features don't seem to be very relevant to the quality.

3. I think there was very low learning rate in the previous study.

4. Is there other training data combination that represents these data well?

first of all, I will adjust learning rate and the number of iteration

In [417]:
gradients = grad(model_loss,argnum = 2)

w = np.zeros(datas_train.shape[1],dtype=float)

b=0.

In [419]:
# lr = 1e-10

# change = np.Inf

# i = 0

# old_val_loss = 1e-15

# while change >= 1e-15 and i<100000:
#     grads = gradients(datas_train, labels_train,(w,b))
#     w-=(grads[0] * lr)
#     b-=(grads[1] * lr)

#     val_loss = model_loss(datas_val,labels_val,(w,b))

#     pred_labels_val = classify(datas_val, (w,b))
#     score = performance(pred_labels_val, labels_val)

#     change = np.abs((val_loss - old_val_loss)/old_val_loss)

#     i += 1
#     old_val_loss = val_loss

#     if i% 500 == 0:
#         print("{}...".format(i),end="")
# print("")
# print("")
# print("Upon optimization stopped:")
# print("    Iterations:", i)
# print("    Validation loss:", val_loss)
# print("    Validation precision:", score[0])
# print("    Validation recall:", score[1])
# print("    Validation F-score:", score[2])
# print("    Validation Accuracy:", score[3])
# print("    Change in validation loss:", change)

500...1000...1500...2000...2500...3000...3500...4000...4500...5000...5500...6000...6500...7000...7500...8000...8500...9000...9500...10000...10500...11000...11500...12000...12500...13000...13500...14000...14500...15000...15500...16000...16500...17000...17500...18000...18500...19000...19500...20000...20500...21000...21500...22000...22500...23000...23500...24000...24500...25000...25500...26000...26500...27000...27500...28000...28500...29000...29500...30000...30500...31000...31500...32000...32500...33000...33500...34000...34500...35000...35500...36000...36500...37000...37500...38000...38500...39000...39500...40000...40500...41000...41500...42000...42500...43000...43500...44000...44500...45000...45500...46000...46500...47000...47500...48000...48500...49000...49500...50000...50500...51000...51500...52000...52500...53000...53500...54000...54500...55000...55500...56000...56500...57000...57500...58000...58500...59000...59500...60000...60500...61000...61500...62000...62500...63000...63500...6400

Upon optimization stopped:
    Iterations: 100000
    Validation loss: 198.6908183917642
    Validation precision: 0.6210526315789474
    Validation recall: 0.5566037735849056
    Validation F-score: 0.5870646766169155
    Validation Accuracy: 0.710801393728223
    Change in validation loss: 1.1041792230517047e-08

I don't think there's much difference.

then what about adjusting lambda?

In [424]:
gradients = grad(model_loss,argnum = 2)

w = np.zeros(datas_train.shape[1],dtype=float)

b=0.

In [425]:
lr = 1e-5

change = np.Inf

i = 0

old_val_loss = 1e-15

while change >= 1e-15 and i<10000:
    grads = gradients(datas_train, labels_train,(w,b))
    w-=(grads[0] * lr)
    b-=(grads[1] * lr)

    val_loss = model_loss(datas_val,labels_val,(w,b),0.000001)

    pred_labels_val = classify(datas_val, (w,b))
    score = performance(pred_labels_val, labels_val)

    change = np.abs((val_loss - old_val_loss)/old_val_loss)

    i += 1
    old_val_loss = val_loss

    if i% 50 == 0:
        print("{}...".format(i),end="")
print("")
print("")
print("Upon optimization stopped:")
print("    Iterations:", i)
print("    Validation loss:", val_loss)
print("    Validation precision:", score[0])
print("    Validation recall:", score[1])
print("    Validation F-score:", score[2])
print("    Validation Accuracy:", score[3])
print("    Change in validation loss:", change)

50...100...150...200...250...300...350...400...450...500...550...600...650...700...750...800...850...900...950...1000...1050...1100...1150...1200...1250...1300...1350...1400...1450...1500...1550...1600...1650...1700...1750...1800...1850...1900...1950...2000...2050...2100...2150...2200...2250...2300...2350...2400...2450...2500...2550...2600...2650...2700...2750...2800...2850...2900...2950...3000...3050...3100...3150...3200...3250...3300...3350...3400...3450...3500...3550...3600...3650...3700...3750...3800...3850...3900...3950...4000...4050...4100...4150...4200...4250...4300...4350...4400...4450...4500...4550...4600...4650...4700...4750...4800...4850...4900...4950...5000...5050...5100...5150...5200...5250...5300...5350...5400...5450...5500...5550...5600...5650...5700...5750...5800...5850...5900...5950...6000...6050...6100...6150...6200...6250...6300...6350...6400...6450...6500...6550...6600...6650...6700...6750...6800...6850...6900...6950...7000...7050...7100...7150...7200...7250...7300.

I don't think there's much difference.

then what about adjusting features?

Let's add a feature whose value is the square of the alcohol feature.

In [430]:
sqalco_datas = pd.read_csv("winequality-red.csv")
sqalco_datas["sqalco"] = [0.] * sqalco_datas.shape[0]
for i in range(sqalco_datas.shape[0]):
    sqalco_datas["sqalco"][i] = sqalco_datas["alcohol"][i] ** 2

print(sqalco_datas.info())


C:\Users\USER\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1439 entries, 0 to 1438
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1439 non-null   float64
 1   volatile acidity      1439 non-null   float64
 2   citric acid           1439 non-null   float64
 3   residual sugar        1439 non-null   float64
 4   chlorides             1439 non-null   float64
 5   free sulfur dioxide   1439 non-null   float64
 6   total sulfur dioxide  1439 non-null   float64
 7   density               1439 non-null   float64
 8   pH                    1439 non-null   float64
 9   sulphates             1439 non-null   float64
 10  alcohol               1439 non-null   float64
 11  quality               1439 non-null   int64  
 12  sqalco                1439 non-null   float64
dtypes: float64(12), int64(1)
memory usage: 146.3 KB
None


In [431]:
print(sqalco_datas.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  sqalco  
0      9.4        5   88.36  
1      9.8        5   96.0

In [432]:
y_col = "quality"
x_cols = sqalco_datas.columns.drop(["quality"])

for i in range(len(sqalco_datas[y_col])):
    if sqalco_datas[y_col][i] < 6:
        sqalco_datas[y_col][i] = 0.0
    else:
        sqalco_datas[y_col][i] = 1.0

sqalco_datas[x_cols] = StandardScaler().fit_transform(sqalco_datas[x_cols])
#
num_of_datas = sqalco_datas.shape[0]
num_of_features = len(x_cols)
print(num_of_datas,num_of_features)

num_of_val = int(num_of_datas * 0.2)

num_of_test = int(num_of_datas * 0.2)

num_of_train = num_of_datas - num_of_val - num_of_test

datas_division = np.split(sqalco_datas[x_cols],[num_of_val, num_of_val + num_of_test],0)
datas_val = datas_division[0].values
datas_test = datas_division[1].values
datas_train = datas_division[2].values

labels_division = np.split(sqalco_datas[y_col],[num_of_val, num_of_val + num_of_test],0)
labels_val = labels_division[0].values
labels_test = labels_division[1].values
labels_train = labels_division[2].values

assert labels_train.shape[0] == datas_train.shape[0]
assert labels_val.shape[0] == datas_val.shape[0]
assert labels_test.shape[0] == datas_test.shape[0]

C:\Users\USER\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\USER\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


1439 12


In [435]:
gradients = grad(model_loss,argnum = 2)

w = np.zeros(datas_train.shape[1],dtype=float)

b=0.

In [436]:
lr = 1e-5

change = np.Inf

i = 0

old_val_loss = 1e-15

while change >= 1e-15 and i<10000:
    grads = gradients(datas_train, labels_train,(w,b))
    w-=(grads[0] * lr)
    b-=(grads[1] * lr)

    val_loss = model_loss(datas_val,labels_val,(w,b))

    pred_labels_val = classify(datas_val, (w,b))
    score = performance(pred_labels_val, labels_val)

    change = np.abs((val_loss - old_val_loss)/old_val_loss)

    i += 1
    old_val_loss = val_loss

    if i% 50 == 0:
        print("{}...".format(i),end="")
print("")
print("")
print("Upon optimization stopped:")
print("    Iterations:", i)
print("    Validation loss:", val_loss)
print("    Validation precision:", score[0])
print("    Validation recall:", score[1])
print("    Validation F-score:", score[2])
print("    Validation Accuracy:", score[3])
print("    Change in validation loss:", change)

50...100...150...200...250...300...350...400...450...500...550...600...650...700...750...800...850...900...950...1000...1050...1100...1150...1200...1250...1300...1350...1400...1450...1500...1550...1600...1650...1700...1750...1800...1850...1900...1950...2000...2050...2100...2150...2200...2250...2300...2350...2400...2450...2500...2550...2600...2650...2700...2750...2800...2850...2900...2950...3000...3050...3100...3150...3200...3250...3300...3350...3400...3450...3500...3550...3600...3650...3700...3750...3800...3850...3900...3950...4000...4050...4100...4150...4200...4250...4300...4350...4400...4450...4500...4550...4600...4650...4700...4750...4800...4850...4900...4950...5000...5050...5100...5150...5200...5250...5300...5350...5400...5450...5500...5550...5600...5650...5700...5750...5800...5850...5900...5950...6000...6050...6100...6150...6200...6250...6300...6350...6400...6450...6500...6550...6600...6650...6700...6750...6800...6850...6900...6950...7000...7050...7100...7150...7200...7250...7300.

I don't think there's much difference.....

what about using pca?

In [438]:
from sklearn import decomposition
pca = decomposition.PCA()

pca_datas = pd.read_csv("winequality-red.csv")

y_col = "quality"
x_cols = pca_datas.columns.drop(["quality"])

for i in range(len(pca_datas[y_col])):
    if pca_datas[y_col][i] < 6:
        pca_datas[y_col][i] = 0.0
    else:
        pca_datas[y_col][i] = 1.0

pca_datas[x_cols] = StandardScaler().fit_transform(pca_datas[x_cols])

pca.n_components = 2
pca_data = pca.fit_transform(pca_datas[x_cols])
print("shape of pca_reduced.shape = ", pca_data.shape)

C:\Users\USER\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\USER\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


shape of pca_reduced.shape =  (1439, 2)


In [441]:
# watch out don't copy this!!!!
num_of_datas = pca_datas.shape[0]
num_of_features = (pca_data.shape[1])
print(num_of_datas,num_of_features)

num_of_val = int(num_of_datas * 0.2)

num_of_test = int(num_of_datas * 0.2)

num_of_train = num_of_datas - num_of_val - num_of_test

datas_division = np.split(pca_data,[num_of_val, num_of_val + num_of_test],0)
datas_val = datas_division[0]
datas_test = datas_division[1]
datas_train = datas_division[2]

labels_division = np.split(pca_datas[y_col],[num_of_val, num_of_val + num_of_test],0)
labels_val = labels_division[0].values
labels_test = labels_division[1].values
labels_train = labels_division[2].values

assert labels_train.shape[0] == datas_train.shape[0]
assert labels_val.shape[0] == datas_val.shape[0]
assert labels_test.shape[0] == datas_test.shape[0]

1439 2


아래 코드는 feature의 갯수를 PCA모듈을

In [442]:
# watch out don't copy this!!!!
gradients = grad(model_loss,argnum = 2)

w = np.zeros(datas_train.shape[1],dtype=float)

b=0.

lr = 1e-5

change = np.Inf

i = 0

old_val_loss = 1e-15

while change >= 1e-15 and i<10000:
    grads = gradients(datas_train, labels_train,(w,b))
    w-=(grads[0] * lr)
    b-=(grads[1] * lr)

    val_loss = model_loss(datas_val,labels_val,(w,b))

    pred_labels_val = classify(datas_val, (w,b))
    score = performance(pred_labels_val, labels_val)

    change = np.abs((val_loss - old_val_loss)/old_val_loss)

    i += 1
    old_val_loss = val_loss

    if i% 50 == 0:
        print("{}...".format(i),end="")
print("")
print("")
print("Upon optimization stopped:")
print("    Iterations:", i)
print("    Validation loss:", val_loss)
print("    Validation precision:", score[0])
print("    Validation recall:", score[1])
print("    Validation F-score:", score[2])
print("    Validation Accuracy:", score[3])
print("    Change in validation loss:", change)

50...100...150...200...250...300...350...400...450...500...550...600...650...700...750...800...850...900...950...1000...1050...1100...1150...1200...1250...1300...1350...1400...1450...1500...1550...1600...1650...1700...1750...1800...1850...1900...1950...2000...2050...2100...2150...2200...2250...2300...2350...2400...2450...2500...2550...2600...2650...2700...2750...2800...2850...2900...2950...3000...3050...3100...3150...3200...3250...3300...3350...3400...3450...3500...3550...3600...3650...3700...3750...3800...3850...3900...3950...4000...4050...4100...4150...4200...4250...4300...4350...4400...4450...4500...4550...4600...4650...4700...4750...4800...4850...4900...4950...5000...5050...5100...5150...5200...5250...5300...5350...5400...5450...5500...5550...5600...5650...5700...5750...5800...5850...5900...5950...6000...6050...6100...6150...6200...6250...6300...6350...6400...6450...6500...6550...6600...6650...6700...6750...6800...6850...6900...6950...7000...7050...7100...7150...7200...7250...7300.

accuracy decresed...

At last, what about data distribution???

let's use sqalco_datas again

아래 코드는 csv파일로 받은 데이터의 행을 셔플해서 학습한 결과입니다.

In [ ]:
#from itertools import permutations as pm

sqalco_datas = pd.read_csv("winequality-red.csv")

sqalco_datas = sqalco_datas.sample(frac=1).reset_index(drop=True)



sqalco_datas["sqalco"] = [0.] * sqalco_datas.shape[0]
for i in range(sqalco_datas.shape[0]):
    sqalco_datas["sqalco"][i] = sqalco_datas["alcohol"][i] ** 2

y_col = "quality"
x_cols = sqalco_datas.columns.drop(["quality"])

for i in range(len(sqalco_datas[y_col])):
    if sqalco_datas[y_col][i] < 6:
        sqalco_datas[y_col][i] = 0.0
    else:
        sqalco_datas[y_col][i] = 1.0

sqalco_datas[x_cols] = StandardScaler().fit_transform(sqalco_datas[x_cols])



num_of_val = int(num_of_datas * 0.2)

num_of_test = int(num_of_datas * 0.2)

num_of_train = num_of_datas - num_of_val - num_of_test

datas_division = np.split(sqalco_datas[x_cols],[num_of_val, num_of_val + num_of_test],0)
datas_val = datas_division[0].values
datas_test = datas_division[1].values
datas_train = datas_division[2].values

labels_division = np.split(sqalco_datas[y_col],[num_of_val, num_of_val + num_of_test],0)
labels_val = labels_division[0].values
labels_test = labels_division[1].values
labels_train = labels_division[2].values

assert labels_train.shape[0] == datas_train.shape[0]
assert labels_val.shape[0] == datas_val.shape[0]
assert labels_test.shape[0] == datas_test.shape[0]

gradients = grad(model_loss,argnum = 2)

w = np.zeros(datas_train.shape[1],dtype=float)

b=0.

lr = 1e-5

change = np.Inf

i = 0

old_val_loss = 1e-15

while change >= 1e-15 and i<10000:
    grads = gradients(datas_train, labels_train,(w,b))
    w-=(grads[0] * lr)
    b-=(grads[1] * lr)

    val_loss = model_loss(datas_val,labels_val,(w,b))

    pred_labels_val = classify(datas_val, (w,b))
    score = performance(pred_labels_val, labels_val)

    change = np.abs((val_loss - old_val_loss)/old_val_loss)

    i += 1
    old_val_loss = val_loss

    if i% 50 == 0:
        print("{}...".format(i),end="")
print("")
print("")
print("Upon optimization stopped:")
print("    Iterations:", i)
print("    Validation loss:", val_loss)
print("    Validation precision:", score[0])
print("    Validation recall:", score[1])
print("    Validation F-score:", score[2])
print("    Validation Accuracy:", score[3])
print("    Change in validation loss:", change)

마지막 최종 코드입니다.

아래 코드는 permutation을 이용해서 k-fold cross validation을 test case도 함께 포함해서 학습한 결과입니다.
추가로 alcohol feature의 제곱을 맨 마지막 열에 하나의 feature로서 추가했습니다.

In [487]:
from itertools import permutations as pm

pm_helper = [0,1,2,3,4]

pm_list = pm(pm_helper,2)

sqalco_datas = pd.read_csv("winequality-red.csv")
sqalco_datas=sqalco_datas.append(sqalco_datas.loc[0],ignore_index = True)

#print(sqalco_datas.shape[0])
#sqalco_datas = sqalco_datas.sample(frac=1).reset_index(drop=True)



sqalco_datas["sqalco"] = [0.] * sqalco_datas.shape[0]

for i in range(sqalco_datas.shape[0]):
    sqalco_datas["sqalco"][i] = sqalco_datas["alcohol"][i] ** 2

y_col = "quality"
x_cols = sqalco_datas.columns.drop(["quality"])

for i in range(len(sqalco_datas[y_col])):
    if sqalco_datas[y_col][i] < 6:
        sqalco_datas[y_col][i] = 0.0
    else:
        sqalco_datas[y_col][i] = 1.0

sqalco_datas[x_cols] = StandardScaler().fit_transform(sqalco_datas[x_cols])




num_of_datas = sqalco_datas.shape[0]
num_of_features = len(x_cols)
#print(num_of_datas,num_of_features)

best_scores = 0.0
best_w = None
best_b = None
best_fsc = 0.0
best_acc = 0.0

datas_division = np.split(sqalco_datas[x_cols],5,0)
labels_division = np.split(sqalco_datas[y_col],5,0)

assert datas_division[0].shape[0] == labels_division[0].shape[0]
assert datas_division[1].shape[0] == labels_division[1].shape[0]
assert datas_division[2].shape[0] == labels_division[2].shape[0]
assert datas_division[3].shape[0] == labels_division[3].shape[0]
assert datas_division[4].shape[0] == labels_division[4].shape[0]
wtt = 0
for iters in pm_list:
    wtt += 1
    pm_helper = [0,1,2,3,4]

    datas_val=datas_division[iters[0]].values
    datas_test = datas_division[iters[1]].values

    labels_val = labels_division[iters[0]].values
    labels_test = labels_division[iters[1]].values

    pm_helper.remove(iters[0])
    pm_helper.remove(iters[1])
    
    datas_train=np.vstack((datas_division[pm_helper[0]].values,datas_division[pm_helper[1]].values,datas_division[pm_helper[2]].values))


    labels_train=np.hstack((labels_division[pm_helper[0]].values,labels_division[pm_helper[1]].values,labels_division[pm_helper[2]].values))

    

    
    #print(labels_train.shape)
    #print(datas_train.shape)
    assert labels_train.shape[0] == datas_train.shape[0]
    assert labels_val.shape[0] == datas_val.shape[0]
    assert labels_test.shape[0] == datas_test.shape[0]

    gradients = grad(model_loss,argnum = 2)

    w = np.zeros(datas_train.shape[1],dtype=float)

    b=0.

    lr = 1e-5

    change = np.Inf

    i = 0

    old_val_loss = 1e-15

    while change >= 1e-15 and i<10000:#
        grads = gradients(datas_train, labels_train,(w,b))
        w-=(grads[0] * lr)
        b-=(grads[1] * lr)

        val_loss = model_loss(datas_val,labels_val,(w,b))

        pred_labels_val = classify(datas_val, (w,b))
        score = performance(pred_labels_val, labels_val)

        change = np.abs((val_loss - old_val_loss)/old_val_loss)

        i += 1
        old_val_loss = val_loss

        if i% 5000 == 0:
            print("{}...".format(i),end="")
        if wtt % 5 == 0:
            print("야후!")
    if ( score[3] + score[2] ) > best_scores:
        best_b = b
        best_w = w
        best_fsc = score[2]
        best_acc = score[3]
        best_scores = score[3] + score[2]
#print("")
#print("")
#print("Upon optimization stopped:")
#print("    Iterations:", i)
#print("    Validation loss:", val_loss)
#print("    Validation precision:", score[0])
#print("    Validation recall:", score[1])
print("    Validation F-score:", score[2])
print("    Validation Accuracy:", score[3])
print("    Change in validation loss:", change)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [488]:
print(best_fsc,best_acc,best_b,best_w)

0.8697674418604651 0.8055555555555556 0.2240490123547769 [ 0.22510538 -0.68556668 -0.2680487  -0.04079619 -0.14484279  0.3138903
 -0.71775561  0.13982565  0.13648678  0.40877563  0.43929911  0.44778318]


0.8697674418604651 0.8055555555555556 0.2240490123547769 [ 0.22510538 -0.68556668 -0.2680487  -0.04079619 -0.14484279  0.3138903
 -0.71775561  0.13982565  0.13648678  0.40877563  0.43929911  0.44778318

In [486]:
tdatas = pd.read_csv("winequality-red_test.csv")
tdatas["sqalco"] = [0.] * tdatas.shape[0]

for i in range(tdatas.shape[0]):
    tdatas["sqalco"][i] = tdatas["alcohol"][i] ** 2

tdatas = StandardScaler().fit_transform(tdatas)
dec = classify(tdatas,(best_w,best_b))

outdatas=pd.read_csv("sample_sub.csv")
for i in range(len(outdatas["quality"])):
    outdatas["quality"][i] = int(dec[i])
outdatas.to_csv("./result2.csv",index=False)


C:\Users\USER\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\USER\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":


In [490]:
sqalco_datas = pd.read_csv("winequality-red.csv")
sqalco_datas=sqalco_datas.append(sqalco_datas.loc[0],ignore_index = True)

#print(sqalco_datas.shape[0])
#sqalco_datas = sqalco_datas.sample(frac=1).reset_index(drop=True)
pm_helper = [0,1,2,3,4]
iters = [3,2]


sqalco_datas["sqalco"] = [0.] * sqalco_datas.shape[0]

for i in range(sqalco_datas.shape[0]):
    sqalco_datas["sqalco"][i] = sqalco_datas["alcohol"][i] ** 2

y_col = "quality"
x_cols = sqalco_datas.columns.drop(["quality"])

for i in range(len(sqalco_datas[y_col])):
    if sqalco_datas[y_col][i] < 6:
        sqalco_datas[y_col][i] = 0.0
    else:
        sqalco_datas[y_col][i] = 1.0

sqalco_datas[x_cols] = StandardScaler().fit_transform(sqalco_datas[x_cols])




num_of_datas = sqalco_datas.shape[0]
num_of_features = len(x_cols)
#print(num_of_datas,num_of_features)

best_scores = 0.0
best_w = None
best_b = None
best_fsc = 0.0
best_acc = 0.0

datas_division = np.split(sqalco_datas[x_cols],5,0)
labels_division = np.split(sqalco_datas[y_col],5,0)

assert datas_division[0].shape[0] == labels_division[0].shape[0]
assert datas_division[1].shape[0] == labels_division[1].shape[0]
assert datas_division[2].shape[0] == labels_division[2].shape[0]
assert datas_division[3].shape[0] == labels_division[3].shape[0]
assert datas_division[4].shape[0] == labels_division[4].shape[0]

datas_val=datas_division[iters[0]].values
datas_test = datas_division[iters[1]].values

labels_val = labels_division[iters[0]].values
labels_test = labels_division[iters[1]].values

pm_helper.remove(iters[0])
pm_helper.remove(iters[1])
    
datas_train=np.vstack((datas_division[pm_helper[0]].values,datas_division[pm_helper[1]].values,datas_division[pm_helper[2]].values))


labels_train=np.hstack((labels_division[pm_helper[0]].values,labels_division[pm_helper[1]].values,labels_division[pm_helper[2]].values))

    

    
#print(labels_train.shape)
#print(datas_train.shape)
assert labels_train.shape[0] == datas_train.shape[0]
assert labels_val.shape[0] == datas_val.shape[0]
assert labels_test.shape[0] == datas_test.shape[0]

gradients = grad(model_loss,argnum = 2)

w = np.zeros(datas_train.shape[1],dtype=float)

b=0.

lr = 1e-3

change = np.Inf

i = 0

old_val_loss = 1e-15

while change >= 1e-15 and i<10000:#
    grads = gradients(datas_train, labels_train,(w,b))
    w-=(grads[0] * lr)
    b-=(grads[1] * lr)

    val_loss = model_loss(datas_val,labels_val,(w,b))

    pred_labels_val = classify(datas_val, (w,b))
    score = performance(pred_labels_val, labels_val)

    change = np.abs((val_loss - old_val_loss)/old_val_loss)

    i += 1
    old_val_loss = val_loss

    if i% 5000 == 0:
        print("{}...".format(i),end="")


print("")
print("")
print("Upon optimization stopped:")
print("    Iterations:", i)
print("    Validation loss:", val_loss)
print("    Validation precision:", score[0])
print("    Validation recall:", score[1])
print("    Validation F-score:", score[2])
print("    Validation Accuracy:", score[3])
print("    Change in validation loss:", change)

5000...

Upon optimization stopped:
    Iterations: 8339
    Validation loss: 131.70616294088418
    Validation precision: 0.8423423423423423
    Validation recall: 0.8904761904761904
    Validation F-score: 0.8657407407407406
    Validation Accuracy: 0.7986111111111112
    Change in validation loss: 8.631854059300477e-16


In [ ]:
tdatas = pd.read_csv("winequality-red_test.csv")
tdatas["sqalco"] = [0.] * tdatas.shape[0]

for i in range(tdatas.shape[0]):
    tdatas["sqalco"][i] = tdatas["alcohol"][i] ** 2

tdatas = StandardScaler().fit_transform(tdatas)
dec = classify(tdatas,(w,b))

outdatas=pd.read_csv("sample_sub.csv")
for i in range(len(outdatas["quality"])):
    outdatas["quality"][i] = int(dec[i])
outdatas.to_csv("./result3.csv",index=False)